In [1]:
import pandas as pd

### Скачаем датасет с которым будем работать

In [2]:
def DownloadCSVTables(url, start, end):
    ""
    # Пустая таблица для последующей конкатенации
    table = pd.DataFrame(columns=['Name', 'Gender', 'Count'])

    # Года записей в таблице
    years = [year for year in range(start, end+1)]    

    # Получим каждую таблицу и добавим в общий DataFrame
    for year in years:
        table_ = pd.read_csv(url + str(year) + '.txt', names=['Name', 'Gender', 'Count'])
        table_['Year'] = str(year)
        
        table = pd.concat([table_, table], sort=False)
        
    return table

data = DownloadCSVTables(url='https://raw.githubusercontent.com/wesm/pydata-book/2nd-edition/datasets/babynames/yob', start=1880, end=2010)
data.to_csv('babynames.csv', index=False)

### Считывание больших данных

In [ ]:
data_iter = pd.read_csv('babynames.csv', chunksize=1000)

def data_prep(df, func):
    """ Отфильтруем данные и сложим в другой файлик
    """
    for chunk in df:
        tmp = func(chunk)
        tmp.to_csv('dataset.csv')
        
def func_filter(data):
    return data.head(5)

data_prep(data_iter, func_filter)

### Понижение размера дата фрейма

In [27]:
# Загрузка данных из файла
data = pd.read_csv('babynames.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1690784 entries, 0 to 1690783
Data columns (total 4 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   Name    1690784 non-null  object
 1   Gender  1690784 non-null  object
 2   Count   1690784 non-null  int64 
 3   Year    1690784 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 51.6+ MB


In [15]:
# Загрузка данных из файла с оптимизацией поля Gender
data = pd.read_csv('babynames.csv', dtype={'Name': object, 'Gender': 'category','Count': 'int32','Year': 'int16'})
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1690784 entries, 0 to 1690783
Data columns (total 4 columns):
 #   Column  Non-Null Count    Dtype   
---  ------  --------------    -----   
 0   Name    1690784 non-null  object  
 1   Gender  1690784 non-null  category
 2   Count   1690784 non-null  int32   
 3   Year    1690784 non-null  int16   
dtypes: category(1), int16(1), int32(1), object(1)
memory usage: 24.2+ MB


### Парсинг даты

In [28]:
# Небольшое преобразование чтобы показать что и как нужно делать
data['Year'] = data['Year'].map(lambda x: str(x)+ '-01-01 00:00:01')
data.to_csv('babynames_time_test.csv', index=False)

In [29]:
%%timeit
# Пусть dateutils сам разберется как парсить дату
data = pd.read_csv('babynames_time_test.csv', parse_dates=['Year'])

1.18 s ± 40.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [30]:
%%timeit
# Говорим ему как разобрать дату
date_parse_func = lambda x: pd.datetime.strptime(x, "%Y-%m-%d %H:%M:%S")
data = pd.read_csv('babynames_time_test.csv', parse_dates=['Year'], date_parser=date_parse_func)

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  


31.4 s ± 236 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Индексирование и join

In [22]:
%%timeit
data.merge(data, on='Name')

18 s ± 148 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [23]:
%%timeit
data.set_index('Name')
data.merge(data, left_index=True, right_index=True)

184 ms ± 3.29 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Векторные операции

In [32]:
%%timeit
data['Count'].map(lambda x: x - x/2)

497 ms ± 19.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [33]:
%%timeit
data['Count'] - data['Count']/2

12.4 ms ± 244 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
